In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import re


from PIL import Image
from sklearn import svm
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import plot_roc_curve
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from nltk import pos_tag
import collections
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import operator
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords, wordnet

import string
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.dummy import DummyClassifier

In [2]:
# pip install emoji --upgrade

In [3]:
import emoji

In [4]:
sw = list(stopwords.words('english'))

In [5]:
type(sw)

list

In [6]:
tweets = pd.read_csv('Data/Tweets.csv')

In [7]:
tweets.isna().sum()

tweet_id                            0
airline_sentiment                   0
airline_sentiment_confidence        0
negativereason                   5462
negativereason_confidence        4118
airline                             0
airline_sentiment_gold          14600
name                                0
negativereason_gold             14608
retweet_count                       0
text                                0
tweet_coord                     13621
tweet_created                       0
tweet_location                   4733
user_timezone                    4820
dtype: int64

In [8]:
tweets['negativereason_gold'].value_counts()

Customer Service Issue                      12
Late Flight                                  4
Can't Tell                                   3
Cancelled Flight                             3
Cancelled Flight\nCustomer Service Issue     2
Late Flight\nCancelled Flight                1
Customer Service Issue\nLost Luggage         1
Customer Service Issue\nCan't Tell           1
Bad Flight                                   1
Flight Attendant Complaints                  1
Late Flight\nLost Luggage                    1
Lost Luggage\nDamaged Luggage                1
Late Flight\nFlight Attendant Complaints     1
Name: negativereason_gold, dtype: int64

In [9]:
tweets['airline_sentiment_gold'].value_counts()

negative    32
positive     5
neutral      3
Name: airline_sentiment_gold, dtype: int64

In [10]:
tweets['retweet_count'].value_counts()

0     13873
1       640
2        66
3        22
4        17
5         5
7         3
6         3
22        2
8         1
32        1
9         1
31        1
18        1
15        1
28        1
44        1
11        1
Name: retweet_count, dtype: int64

In [11]:
tweets['tweet_created'].value_counts()

2015-02-24 09:54:34 -0800    5
2015-02-24 11:43:05 -0800    4
2015-02-23 14:18:58 -0800    3
2015-02-23 06:57:24 -0800    3
2015-02-24 11:38:11 -0800    3
                            ..
2015-02-23 09:43:56 -0800    1
2015-02-21 00:50:34 -0800    1
2015-02-18 08:59:50 -0800    1
2015-02-22 13:52:10 -0800    1
2015-02-19 20:16:40 -0800    1
Name: tweet_created, Length: 14247, dtype: int64

In [12]:
tweets['user_timezone'].value_counts()

Eastern Time (US & Canada)    3744
Central Time (US & Canada)    1931
Pacific Time (US & Canada)    1208
Quito                          738
Atlantic Time (Canada)         497
                              ... 
Kuala Lumpur                     1
Tokyo                            1
Saskatchewan                     1
Canberra                         1
Warsaw                           1
Name: user_timezone, Length: 85, dtype: int64

In [13]:
tweets['name'].value_counts()

JetBlueNews        63
kbosspotter        32
_mhertz            29
otisday            28
throthra           27
                   ..
ImmortalDreamer     1
MDL_LMU             1
ardakuyumcu         1
AuthorKristina      1
jamesferrandini     1
Name: name, Length: 7701, dtype: int64

In [14]:
tweets.drop('negativereason_gold', axis=1, inplace=True)
tweets.drop('airline_sentiment_gold', axis=1, inplace=True)
tweets.drop('retweet_count', axis=1, inplace=True)
tweets.drop('tweet_created', axis=1, inplace=True)
tweets.drop('user_timezone', axis=1, inplace=True)
tweets.drop('name', axis=1, inplace=True)

#current list of columns
list(tweets.columns)

['tweet_id',
 'airline_sentiment',
 'airline_sentiment_confidence',
 'negativereason',
 'negativereason_confidence',
 'airline',
 'text',
 'tweet_coord',
 'tweet_location']

In [15]:
tweets['airline_sentiment_confidence'].value_counts()

1.0000    10445
0.6667       71
0.6632       35
0.6559       30
0.6596       30
          ...  
0.3913        1
0.7273        1
0.6353        1
0.6260        1
0.3544        1
Name: airline_sentiment_confidence, Length: 1023, dtype: int64

In [16]:
tweets['negativereason'].value_counts()

Customer Service Issue         2910
Late Flight                    1665
Can't Tell                     1190
Cancelled Flight                847
Lost Luggage                    724
Bad Flight                      580
Flight Booking Problems         529
Flight Attendant Complaints     481
longlines                       178
Damaged Luggage                  74
Name: negativereason, dtype: int64

In [17]:
tweets['negativereason_confidence'].value_counts()

1.0000    3436
0.0000    1344
0.6667      62
0.6632      33
0.6596      29
          ... 
0.3386       1
0.3577       1
0.3249       1
0.3290       1
0.3255       1
Name: negativereason_confidence, Length: 1410, dtype: int64

In [18]:
tweets['airline'].value_counts()

United            3822
US Airways        2913
American          2759
Southwest         2420
Delta             2222
Virgin America     504
Name: airline, dtype: int64

In [19]:
tweets['tweet_coord'].value_counts()

[0.0, 0.0]                      164
[40.64656067, -73.78334045]       6
[32.91792297, -97.00367737]       3
[40.64646912, -73.79133606]       3
[40.69017276, -73.91646118]       2
                               ... 
[39.1780768, -76.670155]          1
[35.22358885, -80.95061712]       1
[36.11159761, -86.7844525]        1
[33.00605781, -96.76519155]       1
[34.00646846, -118.48131246]      1
Name: tweet_coord, Length: 832, dtype: int64

In [20]:
tweets['tweet_location'].value_counts()

Boston, MA              157
New York, NY            156
Washington, DC          150
New York                127
USA                     126
                       ... 
Crowley, Louisiana        1
Bergen, Norway            1
Land of the ✌️free✌️      1
KFAR                      1
Warsaw                    1
Name: tweet_location, Length: 3081, dtype: int64

In [21]:
# tweets.drop('tweet_id', axis=1, inplace=True)
# tweets['name'].value_counts()
# tweets['name'].value_counts()
# tweets['name'].value_counts()

In [22]:
tweets = tweets[tweets['airline_sentiment_confidence'] > 0.80]

In [23]:
tweets.shape

(10459, 9)

In [24]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10459 entries, 0 to 14638
Data columns (total 9 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   tweet_id                      10459 non-null  int64  
 1   airline_sentiment             10459 non-null  object 
 2   airline_sentiment_confidence  10459 non-null  float64
 3   negativereason                7392 non-null   object 
 4   negativereason_confidence     7396 non-null   float64
 5   airline                       10459 non-null  object 
 6   text                          10459 non-null  object 
 7   tweet_coord                   746 non-null    object 
 8   tweet_location                7007 non-null   object 
dtypes: float64(2), int64(1), object(6)
memory usage: 817.1+ KB


In [25]:
#just in case
def labels_to_num(label):
    if label == 'negative':
        label = 0
    elif label == 'positive':
        label = 1
    else:
        label = 2
    return label

In [26]:
tweets['airline_sentiment_num'] = tweets['airline_sentiment'].map(labels_to_num)

In [27]:
tweets['airline_sentiment_num'].value_counts()

0    7392
2    1550
1    1517
Name: airline_sentiment_num, dtype: int64

In [28]:
##Translate nltk POS to wordnet tags for lemmatizer
def get_wordnet_pos(treebank_tag):
    '''
    Translate nltk POS to wordnet tags
    '''
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [29]:
# from nltk.stem import PorterStemmer

In [31]:
from nltk.tokenize import word_tokenize

In [32]:
lemmatizer = WordNetLemmatizer() #lemmatizing
#STOPWORDS

def process_text(text):
    doc = text.lower()
    doc = re.sub(r'@\w+', '', doc) # Remove @s
    doc = re.sub(r'#', '', doc) # Remove hashtags
    doc = re.sub(r':', ' ', emoji.demojize(doc)) # Turn emojis into words
    doc = re.sub(r'http\S+', '',doc) # Remove URLs
    doc = re.sub(r'\$\S+', 'dollar', doc) # Change dollar amounts to dollar
    doc = re.sub(r'[^a-z0-9\s]', '', doc) # Remove punctuation
    doc = re.sub(r'[0-9]+', 'number', doc) # Change number values to number
    
    doc = word_tokenize(doc)
#     doc = [word for word in doc if word not in sw]
   
    
    doc = pos_tag(doc)

    doc = [(word[0], get_wordnet_pos(word[1])) for word in doc]

    doc = [lemmatizer.lemmatize(word[0], word[1]) for word in doc] 

    return ' '.join(doc)

In [33]:
tweets['processed_text'] = tweets['text'].map(process_text)

In [34]:
tweets['text'][0]

'@VirginAmerica What @dhepburn said.'

In [36]:
tweets['processed_text'][0]

'say'

In [37]:
dummied_airlines = pd.get_dummies(tweets['airline'])

In [38]:
tweets = tweets.join(dummied_airlines)

In [39]:
tweets.columns

Index(['tweet_id', 'airline_sentiment', 'airline_sentiment_confidence',
       'negativereason', 'negativereason_confidence', 'airline', 'text',
       'tweet_coord', 'tweet_location', 'airline_sentiment_num',
       'processed_text', 'American', 'Delta', 'Southwest', 'US Airways',
       'United', 'Virgin America'],
      dtype='object')

## Vectorizing

In [40]:
X = tweets.drop(columns=['tweet_id', 'airline_sentiment_confidence', 'airline',
       'negativereason', 'negativereason_confidence', 'text', 'tweet_coord', 'tweet_location', 'airline_sentiment_num', 'airline_sentiment'], axis =1)
y = tweets['airline_sentiment']

In [41]:
##running our train test split
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state=42,
                                                    test_size=0.20)

In [42]:
#try different parameters
vectorizer = TfidfVectorizer(input='content')

In [43]:
sparse_out = vectorizer.fit_transform(X_train['processed_text'])
tfidf_df_train = pd.DataFrame(data = sparse_out.toarray(),
                        columns = vectorizer.get_feature_names(), index = X_train.index)
tfidf_df_train.head()

X_train.drop(['processed_text'], axis=1, inplace = True)
X_train_vec = X_train.join(tfidf_df_train)

C:\Users\Admin\anaconda3\envs\learn-env\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [44]:
sparse_out1 = vectorizer.transform(X_test['processed_text'])
tfidf_df_test = pd.DataFrame(data = sparse_out1.toarray(),
                        columns = vectorizer.get_feature_names(), index = X_test.index)
tfidf_df_test.head()

X_test.drop(['processed_text'], axis=1, inplace = True)
X_test_vec = X_train.join(tfidf_df_test)

C:\Users\Admin\anaconda3\envs\learn-env\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [45]:
#smoteN

In [46]:
tweets.airline_sentiment.value_counts()

negative    7392
neutral     1550
positive    1517
Name: airline_sentiment, dtype: int64

In [47]:
X_train_vec.shape

(8367, 8743)

In [48]:
X_train.shape

(8367, 6)

### Dummy classifier

In [49]:
##checking our dummy/base model score
dc = DummyClassifier(strategy='stratified')
dc.fit(X_train_vec, y_train)

print(cross_val_score(dc, X_train_vec, y_train).mean())

0.5403348994251951


### Support Vector Classifier

In [50]:
svc = svm.SVC()
svc.fit(X_train_vec, y_train)

print(cross_val_score(svc, X_train_vec, y_train, n_jobs=-4, verbose=3).mean())

[Parallel(n_jobs=-4)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=-4)]: Done   2 out of   5 | elapsed: 10.4min remaining: 15.6min
[Parallel(n_jobs=-4)]: Done   5 out of   5 | elapsed: 10.5min finished


0.8186925525297774


### Naive Bayes

In [51]:
nb = MultinomialNB()
nb.fit(X_train_vec, y_train)
print(cross_val_score(nb, X_train_vec, y_train, n_jobs=-4, verbose=3).mean())


[Parallel(n_jobs=-4)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=-4)]: Done   2 out of   5 | elapsed:    2.3s remaining:    3.5s
[Parallel(n_jobs=-4)]: Done   5 out of   5 | elapsed:    2.3s finished


0.7473398219382833


### Required Configurations and Python Libraries

In [52]:
# pip install tweepy

In [53]:
import tweepy as tw
import json
import time
import datetime
import os
import csv
import pickle

In [54]:
Twitter={}
Twitter['Consumer Key'] = 'iAimqbAdWiBrUfXe0Xmuhlyps'
Twitter['Consumer Secret'] = 'YWUyAsX8sA2VuJCwQv4x3MUhAGmm48VlV0ZTx2yWNWqlGCP9Kj'
Twitter['Access Token'] = '54188780-kRmlqKRHleSb1WBBTfrj4CR57wLxJYuU5qH9piYlq'
Twitter['Access Token Secret'] = 'JXzf6VKWIFhVhAuIYWWQX04jCaHgfgjia77ARpK7v1Z3p'

In [55]:
!pip install twitter

In [56]:
import twitter

auth = twitter.oauth.OAuth(Twitter['Access Token'],
                           Twitter['Access Token Secret'],
                           Twitter['Consumer Key'],
                           Twitter['Consumer Secret'])

twitter_api = twitter.Twitter(auth=auth)

# Nothing to see by displaying twitter_api except that it's now a
# defined variable

print(twitter_api)

In [57]:
q = '#Southwest' 

number = 10

# See https://dev.twitter.com/docs/api/1.1/get/search/tweets

search_results = twitter_api.search.tweets(q=q, count=number)


In [58]:
search_results

{'statuses': [{'created_at': 'Tue Aug 16 20:30:03 +0000 2022',
   'id': 1559638641361801216,
   'id_str': '1559638641361801216',
   'text': "What is Justin Timberlake's favourite Ukrainian river? The Crimea River. #Steep #Southwest #Colorado #River #Lake… https://t.co/SQgDsr6PUF",
   'truncated': True,
   'entities': {'hashtags': [{'text': 'Steep', 'indices': [73, 79]},
     {'text': 'Southwest', 'indices': [80, 90]},
     {'text': 'Colorado', 'indices': [91, 100]},
     {'text': 'River', 'indices': [101, 107]},
     {'text': 'Lake', 'indices': [108, 113]}],
    'symbols': [],
    'user_mentions': [],
    'urls': [{'url': 'https://t.co/SQgDsr6PUF',
      'expanded_url': 'https://twitter.com/i/web/status/1559638641361801216',
      'display_url': 'twitter.com/i/web/status/1…',
      'indices': [115, 138]}]},
   'metadata': {'iso_language_code': 'en', 'result_type': 'recent'},
   'source': '<a href="https://donot.com" rel="nofollow">frmwk2</a>',
   'in_reply_to_status_id': None,
   'in_r

In [59]:
process_text(search_results['statuses'][10]['text'])

IndexError: list index out of range